# Question 3 du projet

In [1]:
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
import warnings
warnings.filterwarnings('ignore')

# Lecture et modification des fichiers csv

In [2]:
testA = pd.read_csv(r"representation.testa.csv"  ,  header=None)
testB = pd.read_csv(r"representation.testb.csv"  ,  header=None)
train = pd.read_csv(r"representation.train.csv"  ,  header=None)

true_labels_a=pd.read_csv(r"true_labels.testa.csv"  ,  header=None)
true_labels_b=pd.read_csv(r"true_labels.testb.csv"  ,  header=None)
true_labels_train=pd.read_csv(r"true_labels.train.csv"  ,  header=None)

In [3]:
def modif_df_2(df):
    dataframe=df
    for k in range(len(dataframe)):
        if dataframe[0][k]=="I-PER":
            dataframe[0][k]=0
        elif dataframe[0][k]=="I-ORG":
            dataframe[0][k]=1
        elif dataframe[0][k]=="I-LOC":
            dataframe[0][k]=2
        else :
            dataframe[0][k]=3
    return dataframe
# complexité linéaire

true_a=modif_df_2(true_labels_a)
true_b=modif_df_2(true_labels_b)
true_train=modif_df_2(true_labels_train)

In [4]:
X_train , Y_train = train, list(true_train[0])
X_test_a , Y_test_a = testA,list(true_a[0])
X_test_b , Y_test_b = testB,list(true_b[0])

# Comparaison des stratégies :

In [8]:
# estimateur utilisé :
logit=LogisticRegression(random_state=0)

In [9]:
one=OneVsOneClassifier(logit)
rest=OneVsRestClassifier(logit)

liste_classifieurs=[rest,one]

In [10]:
one.fit(X_train , Y_train)
rest.fit(X_train , Y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=0, solver='lbfgs',
                                                 tol=0.0001, verbose=0,
                                                 warm_start=False),
                    n_jobs=None)

## Matrices de confusion :

In [11]:
def print_confusion(matrice):
    df=pd.DataFrame(matrice, index = ['classe Personne', 'classe Org', 'classe Lieu','classe rien'],
                    columns = ['prédit Personne', 'prédit Org','predit Lieu','predit rien'])
    print(df,"\n")
    return df

In [12]:
Y_pred_rest_a=rest.predict(X_test_a)
Y_pred_rest_b=rest.predict(X_test_b)
Y_pred_one_a=one.predict(X_test_a)
Y_pred_one_b=one.predict(X_test_b)

In [15]:
print("TEST A : \n")
print("One vs Rest : \n")
df=print_confusion(confusion_matrix(Y_test_a,Y_pred_rest_a))
#print(df.to_latex(index=True)) 
print("One vs One : \n")
df=print_confusion(confusion_matrix(Y_test_a,Y_pred_one_a))
#print(df.to_latex(index=True)) 
print('--'*40,'\n\n')
print("TEST B : \n")
print("One vs Rest : \n")
print_confusion(confusion_matrix(Y_test_b,Y_pred_rest_b))
print("One vs One : \n")
print_confusion(confusion_matrix(Y_test_b,Y_pred_one_b))

TEST A : 

One vs Rest : 

                 prédit Personne  prédit Org  predit Lieu  predit rien
classe Personne              101           1            3           24
classe Org                     0          75            3           15
classe Lieu                    1           3           72            7
classe rien                    4           7            9         1675 

One vs One : 

                 prédit Personne  prédit Org  predit Lieu  predit rien
classe Personne              108           1            2           18
classe Org                     1          76            2           14
classe Lieu                    1           8           66            8
classe rien                    9          13            7         1666 

-------------------------------------------------------------------------------- 


TEST B : 

One vs Rest : 

                 prédit Personne  prédit Org  predit Lieu  predit rien
classe Personne              112           3            2     

,prédit Personne,prédit Org,predit Lieu,predit rien
classe Personne,111,4,3,12
classe Org,2,85,10,14
classe Lieu,1,7,88,13
classe rien,14,5,5,1626


Nous allons utiliser la régression logistique car c'est la méthode ayant les meilleures performances (comme montré en question 1). Il nous faut maintenant voir quelle stratégie est meilleure : One vs One ou One vs Rest ?

In [16]:
def compare_algo(liste_classifieurs,X_train,Y_train,X_test,Y_test):    
    comparaison, temps_train, temps_inf, acc=[],[],[],[]
    for clf in liste_classifieurs:
        t1=time.time()
        clf.fit(X_train , Y_train)
        t2=time.time()
        Y_pred=clf.predict(X_test)
        report=classification_report(Y_test,Y_pred,output_dict=True,target_names=['Personne','Organisation','Lieu','Rien'])
        t3=time.time()
        acc.append(clf.score(X_test,Y_test))
        temps_train.append(t2-t1)
        temps_inf.append(t3-t2)
        comparaison.append(pd.DataFrame(report)['Personne'])
    df_comparaison=pd.DataFrame(comparaison,index=['One vs Rest','One vs One'])
    df_comparaison['temps train (s)']=temps_train
    df_comparaison['temps inference (s)']=temps_inf
    df_comparaison['accuracy']=acc
    return df_comparaison [['accuracy','precision','recall','f1-score','temps train (s)','temps inference (s)']]

### Resultats test A :

In [19]:
df=compare_algo(liste_classifieurs,X_train, Y_train, X_test_a, Y_test_a)
df

,accuracy,precision,recall,f1-score,temps train (s),temps inference (s)
One vs Rest,0.9615,0.952830,0.782946,0.859574,4.598698,0.034907
One vs One,0.9580,0.907563,0.837209,0.870968,4.269580,0.062832


In [23]:
#print(df.to_latex(index=True)) 

### Resultats test B :

In [24]:
df=compare_algo(liste_classifieurs,X_train, Y_train, X_test_b, Y_test_b)
df

,accuracy,precision,recall,f1-score,temps train (s),temps inference (s)
One vs Rest,0.9545,0.875000,0.861538,0.868217,4.636598,0.025932
One vs One,0.9550,0.867188,0.853846,0.860465,3.935476,0.071811


In [25]:
#print(df.to_latex(index=True)) 

On remarque donc sur les deux échantillons de test qu'il est plus difficile d'anonimiser que de faire de l'entity recognition. En effet, l'accuracy est plus élevée que la précision. Cela est très probablement dû au fait qu'il n'y a que 4 classes et que la très grande majorité des token ne sont pas difficilles à classer, ils n'ont  pas d'entité (donc classe rien).

De plus on remarque que la stratégie One vs One est plus rapide que la stratégie One vs Rest (interpretation?) et que les performances de ces deux stratégies sont comparables. 